# 词级别可解释性分析
本项目提供模型的词级别可解释性分析，包括LIME、Integrated Gradient、GradShap 三种分析方法，支持分析微调后模型的预测结果，开发者可以通过更改**数据目录**和**模型目录**在自己的任务中使用此项目进行数据分析。

![image](https://user-images.githubusercontent.com/63761690/192739675-63145d59-23c6-416f-bf71-998fd4995254.png)

## 1.导入Python模块与参数配置
首先我们导入必要的导入必要python模块和设置配置参数，词级别可解释性分析算法支持三种待分析的文本 `INTERPRETER_FILE` 数据文件格式：

**格式一：包括文本、标签、预测结果**
```text
<文本>'\t'<标签>'\t'<预测结果>
...
```

**格式二：包括文本、标签**
```text
<文本>'\t'<标签>
...
```

**格式三：只包括文本**
```text
<文本>
准予原告胡某甲与被告韩某甲离婚。
...
```


In [1]:
import functools
import random
import os
import argparse

import jieba
import numpy as np
from trustai.interpretation import VisualizationTextRecord
from trustai.interpretation import get_word_offset
import paddle
from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorWithPadding
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer


grep: warning: GREP_OPTIONS is deprecated; please use an alias or script
/usr/local/lib/python3.7/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.7/dist-packages/paddlenlp/transformers/image_utils.py:213: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  resample=Image.BILINEAR,
/usr/local/lib/python3.7/dist-packages/paddlenlp/transformers/image_utils.py:379: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  resample=Image.NEAREST,
/usr/local/lib/python3.7/dist-packages/paddlenlp/transformers/ernie_vil/feature_extraction.py:65: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUB

In [2]:
# 预先定义配置参数

# 运行环境，可选"cpu","gpu","gpu:x"(x为gpu编号)
DEVICE = "gpu"
# 数据路径
DATASET_DIR = "../data" 
# 训练模型保存路径
PARAM_PATH = "../checkpoint/" 
# tokenizer使用的最大序列长度，ERNIE模型最大不能超过2048。请根据文本长度选择，通常推荐128、256或512，若出现显存不足，请适当调低这一参数
MAX_LENGTH = 128 
# 批处理大小，请结合显存情况进行调整，若出现显存不足，请适当调低这一参数
BATCH_SIZE = 1 
# 待分析解释的数据
INTERPRETER_FILE = "bad_case.txt"
# 可选 "ig","lime","grad" ,可以根据实际任务效果选择解释器
# "grad":GradShap方法依赖interpretdl
# !pip install interpretdl
INTERPRETER = "ig"
# 分析句子中TOP K关键词，K值
KEY_WORDS_NUM = 5

## 2.读取待分析数据

In [3]:
def read_local_dataset(path):
    """
    Read dataset file
    """
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            items = line.strip().split('\t')
            if items[0] == 'Text':
                continue
            items[0] = items[0][:MAX_LENGTH-2]
            if len(items) == 3:
                yield {'text': items[0], 'label': items[1], 'predict': items[2]}
            elif len(items) == 2:
                yield {'text': items[0], 'label': items[1], 'predict': ''}
            elif len(items) == 1:
                yield {'text': items[0], 'label': '', 'predict': ''}
            else:
                raise ValueError("{} should be in fixed format.".format(path))

def preprocess_function(examples, tokenizer, max_seq_length):
    """
    Preprocess dataset
    """
    result = tokenizer(text=examples["text"], max_seq_len=max_seq_length)
    return result

class LocalDataCollatorWithPadding(DataCollatorWithPadding):
    """
    Convert the  result of DataCollatorWithPadding from dict dictionary to a list
    """

    def __call__(self, features):
        batch = super().__call__(features)
        batch = list(batch.values())
        return batch

In [4]:
paddle.set_device(DEVICE)

# Define model & tokenizer
if os.path.exists(PARAM_PATH):
    model = AutoModelForSequenceClassification.from_pretrained(PARAM_PATH)
    tokenizer = AutoTokenizer.from_pretrained(PARAM_PATH)
else:
    raise ValueError("The {} should exist.".format(PARAM_PATH))

# Prepare & preprocess dataset
interpret_path = os.path.join(DATASET_DIR, INTERPRETER_FILE)


interpret_ds = load_dataset(read_local_dataset, path=interpret_path, lazy=False)
trans_func = functools.partial(preprocess_function,
                                tokenizer=tokenizer,
                                max_seq_length=MAX_LENGTH)

interpret_ds = interpret_ds.map(trans_func)

# Batchify dataset
collate_fn = LocalDataCollatorWithPadding(tokenizer)
interpret_batch_sampler = BatchSampler(interpret_ds,
                                    batch_size=BATCH_SIZE,
                                    shuffle=False)
interpret_data_loader = DataLoader(dataset=interpret_ds,
                                batch_sampler=interpret_batch_sampler,
                                collate_fn=collate_fn)


[2022-09-28 04:51:03,566] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load '../checkpoint/'.
W0928 04:51:03.570216  4827 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0928 04:51:03.575362  4827 gpu_resources.cc:91] device: 0, cuDNN Version: 8.1.
[2022-09-28 04:51:06,542] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load '../checkpoint/'.


## 3.开始数据可解释性分析
数据量较大时，数据分析时间较长，请耐心等待

In [5]:
# Init an interpreter
if INTERPRETER == 'ig':
    from trustai.interpretation.token_level import IntGradInterpreter
    interpreter = IntGradInterpreter(model)
elif INTERPRETER == 'lime':
    from trustai.interpretation.token_level import LIMEInterpreter
    interpreter = LIMEInterpreter(model, unk_id=tokenizer.convert_tokens_to_ids('[UNK]'), pad_id=tokenizer.convert_tokens_to_ids('[PAD]'))
else:
    from trustai.interpretation.token_level import GradShapInterpreter
    interpreter = GradShapInterpreter(model)

# Use interpreter to get the importance scores for all data
print("Start token level interpretion, it will take some time...")
analysis_result = []
for batch in interpret_data_loader:
    analysis_result += interpreter(tuple(batch))

# Add CLS and SEP tags to both original text and standard splited tokens
contexts = []
words = []
for i in range(len(interpret_ds)):
    text = interpret_ds.data[i]["text"]
    contexts.append("[CLS]" + text + "[SEP]")
    words.append(["[CLS]"] + list(jieba.cut(text)) + ["[SEP]"])

# Get the offset map of tokenized tokens and standard splited tokens
print("Start word level alignment, it will take some time...")
ori_offset_maps = []
word_offset_maps = []
for i in range(len(contexts)):
    ori_offset_maps.append(tokenizer.get_offset_mapping(contexts[i]))
    word_offset_maps.append(get_word_offset(contexts[i], words[i]))

align_res = interpreter.alignment(analysis_result, contexts, words, word_offset_maps, ori_offset_maps, special_tokens=["[CLS]", '[SEP]'],rationale_num=KEY_WORDS_NUM)

Start token level interpretion, it will take some time...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.751 seconds.
Prefix dict has been built successfully.
Start word level alignment, it will take some time...


## 4.数据可解释性分析结果可视化
使用用颜色深浅可视化方式代表句子中词对预测结果的重要程度

In [6]:
from IPython.core.display import display, HTML
class Visualization(VisualizationTextRecord):

    def __init__(self, interpret_res, true_label=None, pred_label=None, words=None):
        if words is not None:
            self.words = words
        else:
            self.words = interpret_res.words
        self.pred_label = pred_label if pred_label is not None else ''
        self.true_label = true_label if true_label is not None else ''
        self.key_words = " ".join(set(interpret_res.rationale_tokens))
        word_attributions = interpret_res.word_attributions
        _max = max(word_attributions)
        _min = min(word_attributions)
        self.word_attributions = [(word_imp - _min) / (_max - _min) for word_imp in word_attributions]

    def record_html(self):
        """change all informations to html"""
        return "".join([
            "<tr>",
            self._format_class(self.true_label),
            self._format_class(self.pred_label),
            self._format_class(self.key_words),
            self._format_word_attributions(),
            "<tr>",
        ])
    def _format_class(self, label):
        return '<td align="center"><text style="padding-right:2em"><b>{label}</b></text></td>'.format(label=label)

def visualize_text(text_records):
    """visualize text"""
    html = ["<table width: 100%, align : center>"]
    rows = ["<tr><th>Label</th>"
            "<th>Prediction</th>"
            "<th>Key words</th>"
            "<th>Important visualization</th>"]
    for record in text_records:
        rows.append(record.record_html())
    html.append("".join(rows))
    html.append("</table>")
    html = HTML("".join(html))
    display(html)
    return html.data


def visualize(interpret_res, ds):
    records = []
    for i in range(len(interpret_res)):
        records.append(Visualization(interpret_res[i], true_label=ds.data[i]["label"], pred_label=ds.data[i]["predict"]))
    html = visualize_text(records)
    return html

In [7]:
# process for vbisualize
html = visualize(align_res, interpret_ds)